In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [2]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [3]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [5]:
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [6]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [7]:
'''
['경마장에', '있는'], ['경마장에', '있는', '말이'], ['경마장에', '있는', '말이', '뛰고']
... 이런식의 샘플
'''

"\n['경마장에', '있는'], ['경마장에', '있는', '말이'], ['경마장에', '있는', '말이', '뛰고']\n... 이런식의 샘플\n"

In [8]:
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [9]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [10]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [11]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [12]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [13]:
print(y) # 모든 샘플에 대한 레이블 출력

[ 3  1  4  5  1  7  1  9 10  1 11]


In [14]:
# 원-핫 인코딩
y = to_categorical(y, num_classes=vocab_size)

In [15]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [17]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4851 - accuracy: 0.0000e+00
Epoch 2/200
1/1 - 0s - loss: 2.4735 - accuracy: 0.0000e+00
Epoch 3/200
1/1 - 0s - loss: 2.4619 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4501 - accuracy: 0.1818
Epoch 5/200
1/1 - 0s - loss: 2.4382 - accuracy: 0.2727
Epoch 6/200
1/1 - 0s - loss: 2.4260 - accuracy: 0.2727
Epoch 7/200
1/1 - 0s - loss: 2.4135 - accuracy: 0.2727
Epoch 8/200
1/1 - 0s - loss: 2.4007 - accuracy: 0.2727
Epoch 9/200
1/1 - 0s - loss: 2.3875 - accuracy: 0.4545
Epoch 10/200
1/1 - 0s - loss: 2.3738 - accuracy: 0.4545
Epoch 11/200
1/1 - 0s - loss: 2.3596 - accuracy: 0.4545
Epoch 12/200
1/1 - 0s - loss: 2.3449 - accuracy: 0.4545
Epoch 13/200
1/1 - 0s - loss: 2.3294 - accuracy: 0.4545
Epoch 14/200
1/1 - 0s - loss: 2.3133 - accuracy: 0.4545
Epoch 15/200
1/1 - 0s - loss: 2.2964 - accuracy: 0.4545
Epoch 16/200
1/1 - 0s - loss: 2.2788 - accuracy: 0.4545
Epoch 17/200
1/1 - 0s - loss: 2.2603 - accuracy: 0.4545
Epoch 18/200
1/1 - 0s - loss: 2.2409 - accuracy: 

Epoch 147/200
1/1 - 0s - loss: 0.3051 - accuracy: 0.9091
Epoch 148/200
1/1 - 0s - loss: 0.2993 - accuracy: 0.9091
Epoch 149/200
1/1 - 0s - loss: 0.2936 - accuracy: 0.9091
Epoch 150/200
1/1 - 0s - loss: 0.2880 - accuracy: 0.9091
Epoch 151/200
1/1 - 0s - loss: 0.2826 - accuracy: 0.9091
Epoch 152/200
1/1 - 0s - loss: 0.2773 - accuracy: 0.9091
Epoch 153/200
1/1 - 0s - loss: 0.2721 - accuracy: 0.9091
Epoch 154/200
1/1 - 0s - loss: 0.2670 - accuracy: 0.9091
Epoch 155/200
1/1 - 0s - loss: 0.2620 - accuracy: 0.9091
Epoch 156/200
1/1 - 0s - loss: 0.2572 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.2524 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.2478 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.2433 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.2388 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.2345 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.2303 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.2261 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

In [18]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [19]:
print(sentence_generation(model, t, '경마장에', 4))
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

/Users/formegusto/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


경마장에 있는 말이 뛰고 있다


In [20]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


In [21]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

가는 말이 고와야 오는 말이 곱다
